In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from rtseg.cells.plotting import generate_fork_plot, generate_abins_lbins, get_bulk_init_area
import numpy as np
import sys
from pathlib import Path
from skimage.io import imsave, imread
import zarr
from numcodecs import Zlib
from rtseg.utils.db_ops import read_from_db
from tifffile import imwrite # used for finer compression things
import h5py
import pandas as pd
from os.path import join
%matplotlib qt5

In [3]:
from rtseg.utils.param_io import load_params
from rtseg.utils.disk_ops import read_files

In [4]:
param = load_params(Path('C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Code/realtimeAnalysis/expt_params.yaml'), ref_type='expt')
param.Save.directory = 'C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Code/realtimeAnalysis/'
save_dir = Path(param.Save.directory)
position = 1
channel_no = 1

In [5]:

forks_filenames = [position_dir / Path('forks.csv') for position_dir in list(save_dir.glob('Pos*'))]
dataframes = [pd.read_csv(filename) for filename in forks_filenames]
data = pd.concat(dataframes, ignore_index=True)

areas = data['area'].to_numpy()
lengths = data['length'].to_numpy()
longs = data['normalized_internal_x'].to_numpy()
counts = data['normalization_counts'].to_numpy()

bin_scale = 15
heatmap_threshold = param.Forkplots.heatmap_threshold
pixel_size = param.Forkplots.pixel_size

heatmap, mean_cell_lengths, abins, lbins, (x, y) = generate_fork_plot(areas, lengths, longs, counts,
                bin_scale=bin_scale,
                pixel_size=pixel_size,
                heatmap_threshold=heatmap_threshold)

min_length = 0.44
arb_div_area = 2.5
init_area = get_bulk_init_area(areas, counts, longs, lengths, min_length=min_length, pixel_size=pixel_size, arb_div_area=arb_div_area)


In [6]:
plt.matshow(heatmap, aspect='auto', interpolation='none', extent=[x[0], x[-1], y[-1], y[0]], origin='upper')
plt.plot(-0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.plot(0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.axhline(init_area, color='red', linestyle='--', linewidth=2)
plt.xlim(-3, 3) 
plt.ylim(3,y[0])
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm$^2$)')

Text(0, 0.5, 'Cell size (µm$^2$)')

In [7]:
print(lbins)

[-9.21508057 -9.1480618  -9.08104303 -9.01402426 -8.9470055  -8.87998673
 -8.81296796 -8.74594919 -8.67893043 -8.61191166 -8.54489289 -8.47787412
 -8.41085535 -8.34383659 -8.27681782 -8.20979905 -8.14278028 -8.07576152
 -8.00874275 -7.94172398 -7.87470521 -7.80768644 -7.74066768 -7.67364891
 -7.60663014 -7.53961137 -7.47259261 -7.40557384 -7.33855507 -7.2715363
 -7.20451753 -7.13749877 -7.07048    -7.00346123 -6.93644246 -6.8694237
 -6.80240493 -6.73538616 -6.66836739 -6.60134862 -6.53432986 -6.46731109
 -6.40029232 -6.33327355 -6.26625479 -6.19923602 -6.13221725 -6.06519848
 -5.99817971 -5.93116095 -5.86414218 -5.79712341 -5.73010464 -5.66308588
 -5.59606711 -5.52904834 -5.46202957 -5.3950108  -5.32799204 -5.26097327
 -5.1939545  -5.12693573 -5.05991697 -4.9928982  -4.92587943 -4.85886066
 -4.7918419  -4.72482313 -4.65780436 -4.59078559 -4.52376682 -4.45674806
 -4.38972929 -4.32271052 -4.25569175 -4.18867299 -4.12165422 -4.05463545
 -3.98761668 -3.92059791 -3.85357915 -3.78656038 -3.7

#### Slicing fork plot to around initiation

In [8]:
#plt.close('all')
init_area_cv = 0.11
init_area_low = init_area - init_area_cv*init_area
init_area_high = init_area + init_area_cv*init_area

abins_inds_around_init = np.where(np.logical_and(init_area_low <= abins, abins <= init_area_high))
abins_inds_around_init = abins_inds_around_init[0]
abins_inds_around_init = np.append(abins_inds_around_init, abins_inds_around_init[-1]+1)
area_bins_around_init = abins[abins_inds_around_init]
mean_cell_length_around_init = mean_cell_lengths[abins_inds_around_init[0:-1]]
max_cell_length_around_init = np.max(mean_cell_length_around_init)
lbins_inds_around_init = np.where(np.logical_and(-0.5*max_cell_length_around_init <= lbins , lbins <= 0.5*max_cell_length_around_init))
lbins_inds_around_init = lbins_inds_around_init[0]
lbins_inds_around_init = np.insert(lbins_inds_around_init, 0, [lbins_inds_around_init[0]-2, lbins_inds_around_init[0]-1])
lbins_inds_around_init = np.append(lbins_inds_around_init, [lbins_inds_around_init[-1]+1, lbins_inds_around_init[-1]+2])
lbins_around_init = lbins[lbins_inds_around_init]

heatmap_around_init = heatmap[np.ix_(abins_inds_around_init, lbins_inds_around_init)]

plot_abins_init = (area_bins_around_init[0:-1] + area_bins_around_init[1:])/2
plt.matshow(heatmap_around_init, aspect='auto', interpolation='none', 
            extent=[lbins_around_init[0], lbins_around_init[-1], area_bins_around_init[-1], area_bins_around_init[0]], origin='upper')
plt.plot(-0.5 * mean_cell_length_around_init, y[abins_inds_around_init[0:-1]], 'w', linewidth=2)
plt.plot(0.5 * mean_cell_length_around_init, y[abins_inds_around_init[0:-1]], 'w', linewidth=2)
plt.axhline(init_area, color='red', linestyle='--', linewidth=2)
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm$^2$)')

Text(0, 0.5, 'Cell size (µm$^2$)')

In [12]:
print(type(lbins_inds_around_init))
print(type(abins_inds_around_init))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


#### Getting a fork plot slice from the single trap fork plot

In [10]:
forks_filename = save_dir / Path('Pos' + str(position)) / Path('forks.csv')
data = pd.read_csv(forks_filename)


# filter for trap no
trap_data = data[data['trap'] == channel_no]

areas = trap_data['area'].to_numpy()
lengths = trap_data['length'].to_numpy()
longs = trap_data['normalized_internal_x'].to_numpy()
counts = trap_data['normalization_counts'].to_numpy()
heatmap_trap, mean_cell_lengths_trap, _, _, _ = generate_fork_plot(areas, lengths, longs, counts, abins=abins, lbins=lbins,
                bin_scale=bin_scale,
                pixel_size=pixel_size,
                heatmap_threshold=heatmap_threshold)



In [11]:
heatmap_trap_init = heatmap_trap[np.ix_(abins_inds_around_init, lbins_inds_around_init)]
mean_cell_lengths_trap_init = mean_cell_lengths_trap[abins_inds_around_init[0:-1]]

plt.matshow(heatmap_trap_init, aspect='auto', interpolation='none', 
            extent=[lbins_around_init[0], lbins_around_init[-1], area_bins_around_init[-1], area_bins_around_init[0]], origin='upper')
plt.plot(-0.5 * mean_cell_lengths_trap_init, y[abins_inds_around_init[0:-1]], 'w', linewidth=2)
plt.plot(0.5 * mean_cell_lengths_trap_init, y[abins_inds_around_init[0:-1]], 'w', linewidth=2)
plt.axhline(init_area, color='red', linestyle='--', linewidth=2)
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm$^2$)')


Text(0, 0.5, 'Cell size (µm$^2$)')

#### Debuggin single trap forkplot

In [61]:

bin_scale = param.Forkplots.bin_scale
heatmap_threshold = param.Forkplots.heatmap_threshold
pixel_size = param.Forkplots.pixel_size


all_forks_filenames = [position_dir / Path('forks.csv') for position_dir in list(save_dir.glob('Pos*'))]
dataframes = [pd.read_csv(filename) for filename in all_forks_filenames]
all_data = pd.concat(dataframes, ignore_index=True)

abins, lbins = generate_abins_lbins(all_data['area'].to_numpy(),
                        all_data['length'].to_numpy(),
                        all_data['normalized_internal_x'].to_numpy(),
                        pixel_size=pixel_size,
                        bin_scale=bin_scale)

print(abins.shape) 
print(lbins.shape) 

(370,)
(370,)


In [34]:

forks_filename = save_dir / Path('Pos' + str(position)) / Path('forks.csv')
data = pd.read_csv(forks_filename)


# filter for trap no
trap_data = data[data['trap'] == channel_no]

areas = trap_data['area'].to_numpy()
lengths = trap_data['length'].to_numpy()
longs = trap_data['normalized_internal_x'].to_numpy()
counts = trap_data['normalization_counts'].to_numpy()
heatmap, mean_cell_lengths, extent = generate_fork_plot(areas, lengths, longs, counts, abins=abins, lbins=lbins,
                bin_scale=bin_scale,
                pixel_size=pixel_size,
                heatmap_threshold=heatmap_threshold)


C:\Users\oscbr226\Desktop\Initiation_Lib_Tweezer\Code\realtimeAnalysis\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [35]:
mean_cell_lengths

array([1.23947163, 1.07760388, 1.01626538, 1.11041656, 1.2924282 ,
       1.28687975, 1.4333637 , 1.46434401, 1.2179682 , 1.49853767,
       1.55044694, 1.45165156, 1.53293816, 1.58808346, 1.55709633,
       1.59829198, 1.76251148, 1.65741037, 1.73405509, 1.70148784,
       1.75349614, 1.85064651, 1.80308845, 1.92399042, 1.92726053,
       1.99268495, 1.95117048, 1.95416739, 2.07666608, 2.07678579,
       2.13770196, 2.18045057, 2.22605986, 2.27963872, 2.2987699 ,
       2.3375184 , 2.36987367, 2.42830591, 2.45106556, 2.5591727 ,
       2.66995233, 2.60935588, 2.63037579, 2.76433842, 2.78237184,
       2.82330265, 2.91427575, 2.92250729, 3.00263082, 3.00088817,
       3.04044379, 3.07131727, 3.14181206, 3.272286  , 3.21861925,
       3.27401881, 3.41680744, 3.28777772, 3.41565973, 3.40830752,
       3.47606152, 3.52568145, 3.5703132 , 3.67766986, 3.66142326,
       3.74163873, 3.74628934, 3.77530204, 3.85525881, 3.85653324,
       3.86229918, 3.89844996, 3.80101465, 4.04134126, 4.17514

In [36]:

plt.matshow(heatmap, aspect='auto', interpolation='none', extent=[x[0], x[-1], y[-1], y[0]], origin='upper')
plt.plot(-0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.plot(0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm^2)')

ValueError: x and y must have same first dimension, but have shapes (369,) and (183,)

In [5]:
d = read_files('single_trap_data_forks', param, 1, 1)

/home/pk/Documents/rtclient/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [6]:
save_dir = param.Save.directory
position = 1
channel_no = 1
forks_filename = save_dir / Path('Pos' + str(position)) / Path('forks.csv')
data = pd.read_csv(forks_filename)

# filter for trap no
trap_data = data[data['trap'] == channel_no]
print(trap_data.shape)

areas = trap_data['area']
lengths = trap_data['length'] 
longs = trap_data['normalized_internal_x']
counts = trap_data['normalization_counts']

(46, 12)


In [12]:
areas.to_numpy()

array([ 674.,  674.,  674.,  978.,  978.,  978.,  978.,  899.,  899.,
        899.,  899.,  847.,  847.,  847.,  847.,  517.,  616.,  616.,
        616., 1755., 1755., 1755., 1755., 1755.,  498.,  498., 1043.,
       1043., 1043., 1043., 1043., 1043., 1043., 1043., 2638., 2638.,
       2638., 2638., 2638., 2638., 2638., 2638., 2638.,  624.,  624.,
        624.])

In [7]:

bin_scale = param.Forkplots.bin_scale
heatmap_threshold = param.Forkplots.heatmap_threshold
pixel_size = param.Forkplots.pixel_size

heatmap, mean_cell_lengths, extent = generate_fork_plot(areas, lengths, longs, counts,
                bin_scale=bin_scale,
                pixel_size=pixel_size,
                heatmap_threshold=heatmap_threshold)


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45]


KeyError: '[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45] not in index'

#### Making forks

In [18]:
areas = data['area']
lengths = data['length'] 
bin_scale=20
longs = data['normalized_internal_x']
counts = data['normalization_counts']

In [22]:
heatmap, mean_cell_lengths, (x, y) = generate_fork_plot(areas, lengths, longs, counts)

plt.matshow(heatmap, aspect='auto', interpolation='none', extent=[x[0], x[-1], y[-1], y[0]], origin='upper')
plt.plot(-0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.plot(0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm^2)')

/home/pk/Documents/rtseg/rtseg/cells/plotting.py:39: RuntimeWarning: invalid value encountered in divide
  heatmap[i, :] = h1 / normFactor


Text(0, 0.5, 'Cell size (µm^2)')

In [26]:
df

[autoreload of rtseg.utils.disk_ops failed: Traceback (most recent call last):
  File "/home/pk/Documents/rtclient/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/pk/Documents/rtclient/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1074, in get_code
  File "<frozen importlib._bootstrap_external>", line 1004, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/pk/Documents/rtseg/rtseg/utils/disk_ops.py", line 391
    heatmap, mean_cell_leng

,position,timepoint,trap,cell_label,area,length,normalization_counts,internal_coord,normalized_internal_x,bbox,global_coords,local_coords
15,7,0,0,20,969.0,77.473008,4,"(5.500170787017906, 3.5189557537008094)",0.070995,"(2898, 231, 2918, 309)","(2901.8526821457167, 236.8943154523619)","(3.852682145716699, 5.894315452361894)"
16,7,0,0,20,969.0,77.473008,4,"(14.108218968953976, 1.8457351293230309)",0.182105,"(2898, 231, 2918, 309)","(2903.4001641458553, 245.603040606558)","(5.400164145855342, 14.603040606558011)"
17,7,0,0,20,969.0,77.473008,4,"(21.799212241954013, -0.951558413839998)",0.281378,"(2898, 231, 2918, 309)","(2906.297166551486, 253.2683713430085)","(8.297166551486043, 22.26837134300851)"
18,7,0,0,20,969.0,77.473008,4,"(55.047599448915, -1.7986868457671277)",0.710539,"(2898, 231, 2918, 309)","(2909.8350542584676, 286.1689575797435)","(11.835054258467608, 55.168957579743505)"
89,7,0,0,55,808.0,72.518836,2,"(16.15227764349672, -3.1853089097937395)",0.222732,"(2901, 312, 2919, 385)","(2909.935291469728, 328.4231375196939)","(8.935291469727872, 16.42313751969391)"
...,...,...,...,...,...,...,...,...,...,...,...,...
14158,10,0,0,296,1078.0,91.342440,3,"(74.9227886635327, -4.5360555901594575)",0.820241,"(2918, 981, 2938, 1073)","(2933.959194390387, 1054.9233002810984)","(15.959194390386983, 73.92330028109836)"
14272,10,0,0,325,1456.0,109.090933,4,"(16.983020534651136, 0.6693342098731568)",0.155678,"(2921, 1070, 2941, 1188)","(2928.4945104098892, 1092.49365647365)","(7.494510409889244, 22.493656473649935)"
14273,10,0,0,325,1456.0,109.090933,4,"(51.203831270031024, 1.5263613945238688)",0.469368,"(2921, 1070, 2941, 1188)","(2928.569807385597, 1126.7437430990062)","(7.56980738559696, 56.743743099006224)"
14274,10,0,0,325,1456.0,109.090933,4,"(93.70535803111275, 0.32005280560508786)",0.858966,"(2921, 1070, 2941, 1188)","(2931.7585286265203, 1169.1617472560072)","(10.758528626520274, 99.16174725600717)"


### Debugging dots on plots

In [5]:
ans = read_files('fluor', param, 1, 0, max_imgs=20)

In [6]:
ans['dots'].shape

(46, 2)

In [7]:
fig, ax = plt.subplots()
ax.imshow(ans['image'], cmap='gray')
ax.plot(ans['dots'][:, 1], ans['dots'][:, 0], 'ro')
plt.show()

In [6]:
position = 1
channel_no = 0
max_imgs=None

In [7]:
save_dir = Path(param.Save.directory)

In [8]:
fluor_dir = save_dir / Path('Pos' + str(position)) / Path('fluor')

# use phase image to update the barcodes
phase_dir = save_dir / Path('Pos' + str(position)) / Path('phase')
# read all files 
fluor_filenames = sorted(list(fluor_dir.glob('*.tiff')))

last_key = int(fluor_filenames[-1].name.split('_')[-1].split('.')[0])

last_fluor_img = imread(fluor_filenames[-1])
height, _ = last_fluor_img.shape

last_phase_img_filename = phase_dir / Path('phase_' + str(last_key).zfill(4) + '.tiff')
last_phase_img = imread(last_phase_img_filename)

barcode_data = read_from_db('barcode_locations', save_dir, position=position, timepoint=last_key)
channel_location = barcode_data['trap_locations'][channel_no]

for i, barcode in enumerate(barcode_data['barcode_locations'], 0):
    if (((barcode[0] + barcode[2])/2) > channel_location):
        break
left_barcode =  barcode_data['barcode_locations'][i-1]
right_barcode = barcode_data['barcode_locations'][i]
left_barcode_img = last_phase_img[int(left_barcode[1]): int(left_barcode[3]), int(left_barcode[0]): int(left_barcode[2])]
right_barcode_img = last_phase_img[int(right_barcode[1]): int(right_barcode[3]), int(right_barcode[0]): int(right_barcode[2])]

if max_imgs is not None:
    files_to_iter = fluor_filenames[-max_imgs:]
else:
    files_to_iter = fluor_filenames


trap_width = param.Save.trap_width
full_img = np.zeros((height, len(files_to_iter) * trap_width))
for i, filename in enumerate(files_to_iter, 0):
    fluor_slice = imread(filename)[:, channel_location - (trap_width//2) : channel_location + (trap_width//2)]
    full_img[:, (i) * trap_width : (i+1) * trap_width] = fluor_slice


dots_filename = save_dir / Path('Pos' + str(position)) / Path('dots.hdf5')

In [31]:
files_to_iter

[PosixPath('/home/pk/Documents/REALTIME/analysis/Pos1/fluor/fluor_0000.tiff')]

In [35]:
keys = []
for filename in files_to_iter:
    keys.append(str(int(filename.stem.split('_')[-1])) + '/raw_coords')

In [36]:
keys

['0/raw_coords']

In [37]:
dot_data = []
with h5py.File(dots_filename, 'r') as f:
    for key in keys:
        dot_data.append(f[key][:])

In [38]:
dot_data

[array([[ 783.14738426, 1487.63868987],
        [1265.01042238, 3027.50923386],
        [ 277.35200941, 2939.82150615],
        ...,
        [1014.54571118,  100.4640608 ],
        [ 168.5046111 ,   19.1091004 ],
        [ 169.431315  ,    9.47310662]])]

### filter dots by trap and recenter them

In [41]:
trap_no, trap_width, channel_location

(0, 80, 116)

In [54]:
dot_array = dot_data[0]

In [55]:
indices = np.where(np.logical_and(dot_array[:, 1] > channel_location - (trap_width//2), dot_array[:, 1] < channel_location + (trap_width//2)))[0]

In [58]:
dot_array_filtered = dot_array[indices]
dot_array_filtered[:, 1] = dot_array_filtered[:, 1] - channel_location + trap_width//2 + 

In [59]:
dot_array_filtered

array([[ 172.71778214,   77.59291434],
       [ 207.55440261,   58.97895049],
       [ 236.89431545,   57.14731785],
       [ 143.15366082,   55.82867086],
       [ 245.60304061,   55.59983585],
       [ 208.37987988,   54.27460794],
       [ 403.55878354,   53.03456172],
       [ 253.26837134,   52.70283345],
       [ 410.68552497,   52.3137105 ],
       [ 479.97305046,   50.51863532],
       [ 286.16895758,   49.16494574],
       [ 328.42313752,   49.06470853],
       [ 419.91564958,   47.59391104],
       [ 141.03306924,   47.532897  ],
       [ 369.98689098,   45.61137208],
       [ 442.62116784,   45.42203885],
       [ 498.96095508,   45.51922889],
       [ 523.43689718,   43.70371778],
       [ 536.47645951,   45.96445386],
       [ 583.31733754,   45.3842334 ],
       [ 150.1277081 ,   45.48802737],
       [ 620.7453464 ,   44.87417775],
       [ 549.64763835,   42.29848213],
       [ 700.50311526,   43.03333333],
       [ 769.33667655,   39.84192463],
       [ 659.57351324,   

In [60]:
a = [dot_array_filtered, dot_array_filtered]

In [61]:
dot_array_filtered.shape

(46, 2)

In [63]:
np.vstack(a).shape

(92, 2)